In [1]:
from datetime import datetime, timedelta
import os
import io
from dotenv import load_dotenv
import requests
import boto3
import pandas as pd

In [14]:
# retrieve the data from the API
url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
response = requests.get(url, timeout=1000)
if response.status_code == 200:
    data = response.json()["results"]
    pokemon_list = []
    for pokemon in data:
        pokemon_url = pokemon["url"]
        response = requests.get(pokemon_url, timeout=1000)
        if response.status_code == 200:
            pokemon_data = response.json()
           
            for bs in pokemon_data["stats"]:
                base_stats = bs["base_stat"]
                base_stats_names = bs["stat"]["name"]
                
                pokemon_dict = {
                "id": pokemon_data["id"],
                "pokemon_name": pokemon_data["name"],
                "base_stat_name": base_stats_names,
                "base_stat": base_stats,
                }
                pokemon_list.append(pokemon_dict)

else:
    print(f"Error: HTTP status code {response.status_code}")

In [16]:

# create a DataFrame from the data
df = pd.DataFrame(pokemon_list)

# explode the "base_stats" column and select relevant columns
#df = df.explode("base_stat_name")
#df = df.explode("base_stat")

df.head(10)

,id,pokemon_name,base_stat_name,base_stat
0,1,bulbasaur,hp,45
1,1,bulbasaur,attack,49
2,1,bulbasaur,defense,49
3,1,bulbasaur,special-attack,65
4,1,bulbasaur,special-defense,65
5,1,bulbasaur,speed,45
6,2,ivysaur,hp,60
7,2,ivysaur,attack,62
8,2,ivysaur,defense,63
9,2,ivysaur,special-attack,80
